In [3]:
import numpy as np

In [4]:
# initialize tensors
d = 2
chi = 10

A = np.random.rand(chi, d, chi)
B = np.random.rand(chi, d, chi)
sAB = np.ones(chi) / np.sqrt(chi)  # set trivial initial weights
sBA = np.ones(chi) / np.sqrt(chi)  # set trivial initial weights

In [7]:
from numpy import linalg as LA
from ncon import ncon

""" Contract infinite MPS from the left for environment tensor sigBA """

sigBA = np.random.rand(chi, chi)  # initialize random starting point
tol = 1e-10  # set convergence tolerance

# define the tensor network
tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
           np.diag(sAB), B, B.conj()]
labels = [[1,2],[1,3],[2,4],[3,5,6],[4,5,7],[6,8],[7,9],[8,10,-1],[9,10,-2]]

for k in range(1000):
    sigBA_new = ncon([sigBA, *tensors], labels)  # contract transfer operator
    sigBA_new = sigBA_new / np.trace(sigBA_new)  # normalize
    if LA.norm(sigBA - sigBA_new) < tol:  # check convergence
        print('success!')
        break
    else:
        print('iter: %d, diff: %e' % (k, LA.norm(sigBA - sigBA_new)))
        sigBA = sigBA_new

iter: 0, diff: 4.765514e+00
iter: 1, diff: 4.360892e-03
iter: 2, diff: 9.112289e-05
iter: 3, diff: 6.716472e-07
iter: 4, diff: 7.091927e-09
iter: 5, diff: 1.148429e-10
success!


In [19]:
from scipy.sparse.linalg import LinearOperator

In [20]:
from scipy.sparse.linalg import eigs

In [21]:
def left_contract_MPS(sigBA, sBA, A, sAB, B):
    """ Contract an infinite 2-site unit cell from the left for the environment
    density matrices sigBA (B-A link) and sigAB (A-B link)"""

    # initialize the starting vector
    chiBA = A.shape[0]
    if sigBA.shape[0] == chiBA:
        v0 = sigBA.reshape(np.prod(sigBA.shape))
    else:
        v0 = (np.eye(chiBA) / chiBA).reshape(chiBA**2)

    # define network for transfer operator contract
    tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
             np.diag(sAB), B, B.conj()]
    labels = [[1, 2], [1, 3], [2, 4], [3, 5, 6], [4, 5, 7], [6, 8], [7, 9],
            [8, 10, -1], [9, 10, -2]]

    # define function for boundary contraction and pass to eigs
    def left_iter(sigBA):
        return ncon([sigBA.reshape([chiBA, chiBA]), *tensors], labels).reshape([chiBA**2, 1])

    # returns k=1 eigenvalues and eigenvectors
    # matvec implements A * v
    Dtemp, sigBA = eigs(LinearOperator((chiBA**2, chiBA**2), matvec=left_iter),
                      k=1, which='LM', v0=v0, tol=1e-10)

    # normalize the environment density matrix sigBA
    if np.isrealobj(A):
        sigBA = np.real(sigBA)
    sigBA = sigBA.reshape(chiBA, chiBA)
    sigBA = 0.5 * (sigBA + np.conj(sigBA.T))
    sigBA = sigBA / np.trace(sigBA)

    # compute density matric sigAB for A-B link
    sigAB = ncon([sigBA, np.diag(sBA), np.diag(sBA), A, np.conj(A)],
               [[1, 2], [1, 3], [2, 4], [3, 5, -1], [4, 5, -2]])
    sigAB = sigAB / np.trace(sigAB)

    return sigBA, sigAB

In [33]:
sigBA = np.random.rand(chi, chi)

In [34]:
sigBA

array([[0.23125688, 0.50475538, 0.60173076, 0.56908957, 0.89367339,
        0.31586685, 0.86391404, 0.43108712, 0.48481998, 0.9296994 ],
       [0.66182162, 0.21266233, 0.12548054, 0.62488701, 0.53620891,
        0.76315266, 0.7507128 , 0.99229489, 0.49106256, 0.49131804],
       [0.52540786, 0.51968261, 0.59491677, 0.21594739, 0.44028092,
        0.21754158, 0.34377785, 0.11859377, 0.21559215, 0.75683459],
       [0.84733102, 0.9302387 , 0.34270982, 0.83610443, 0.37105366,
        0.27189723, 0.70179671, 0.64919382, 0.832932  , 0.33049437],
       [0.38152476, 0.28374764, 0.49450243, 0.75161868, 0.10393255,
        0.53030373, 0.03149392, 0.05196309, 0.83587811, 0.19471896],
       [0.79658456, 0.47629031, 0.86532851, 0.62058257, 0.3798903 ,
        0.80051672, 0.05091785, 0.73971178, 0.67609225, 0.71288487],
       [0.79375644, 0.76244815, 0.46391765, 0.93703762, 0.60302601,
        0.19957118, 0.99236746, 0.83961963, 0.76085248, 0.92130164],
       [0.40826099, 0.25191334, 0.3998379

In [35]:
sigBA, sigAB = left_contract_MPS(sigBA, sBA, A, sAB, B)

In [36]:
sigBA

array([[0.08652234, 0.09648405, 0.0741024 , 0.09485503, 0.10399168,
        0.09083183, 0.07345176, 0.08878771, 0.09719843, 0.08862878],
       [0.09648405, 0.11687967, 0.08852479, 0.11390091, 0.11962155,
        0.11489768, 0.08759954, 0.10420265, 0.11649972, 0.10781385],
       [0.0741024 , 0.08852479, 0.06731012, 0.08649768, 0.09155268,
        0.08648621, 0.06658654, 0.07946011, 0.08839158, 0.08172139],
       [0.09485503, 0.11390091, 0.08649768, 0.11120432, 0.11737302,
        0.11155316, 0.08558325, 0.101997  , 0.1136639 , 0.10514298],
       [0.10399168, 0.11962155, 0.09155268, 0.11737302, 0.12670416,
        0.11462506, 0.09063373, 0.10895692, 0.12001818, 0.110248  ],
       [0.09083183, 0.11489768, 0.08648621, 0.11155316, 0.11462506,
        0.11535701, 0.08552193, 0.10087031, 0.11394883, 0.10628712],
       [0.07345176, 0.08759954, 0.06658654, 0.08558325, 0.09063373,
        0.08552193, 0.06593363, 0.078611  , 0.08751931, 0.08084239],
       [0.08878771, 0.10420265, 0.0794601

In [37]:
sigAB

array([[0.08356492, 0.07982749, 0.09980777, 0.09331453, 0.08906794,
        0.06799676, 0.09030514, 0.0742854 , 0.09482042, 0.09675184],
       [0.07982749, 0.08808157, 0.10472799, 0.09481882, 0.09065625,
        0.07918956, 0.09532697, 0.08278387, 0.09811922, 0.10668281],
       [0.09980777, 0.10472799, 0.12676624, 0.11593568, 0.11080359,
        0.0926275 , 0.11518329, 0.09825364, 0.11929399, 0.12695152],
       [0.09331453, 0.09481882, 0.11593568, 0.10693763, 0.10213721,
        0.08274064, 0.10516473, 0.08860132, 0.10949293, 0.11486621],
       [0.08906794, 0.09065625, 0.11080359, 0.10213721, 0.09756425,
        0.07918053, 0.10052397, 0.08474877, 0.10462262, 0.10983462],
       [0.06799676, 0.07918956, 0.0926275 , 0.08274064, 0.07918053,
        0.07259052, 0.08453028, 0.07483371, 0.0862912 , 0.09598026],
       [0.09030514, 0.09532697, 0.11518329, 0.10516473, 0.10052397,
        0.08453028, 0.10469547, 0.08951337, 0.10831879, 0.11557189],
       [0.0742854 , 0.08278387, 0.0982536

In [38]:
def orthog_MPS(sigBA, muBA, B, sBA, A, dtol=1e-12):
    """ set the MPS gauge across B-A link to the canonical form """

    # diagonalize left environment
    dtemp, utemp = LA.eigh(sigBA)
    chitemp = sum(dtemp > dtol)
    DL = dtemp[range(-1, -chitemp - 1, -1)]
    UL = utemp[:, range(-1, -chitemp - 1, -1)]

    # diagonalize right environment
    dtemp, utemp = LA.eigh(muBA)
    chitemp = sum(dtemp > dtol)
    DR = dtemp[range(-1, -chitemp - 1, -1)]
    UR = utemp[:, range(-1, -chitemp - 1, -1)]

    # compute new weights for B-A link
    weighted_mat = (np.diag(np.sqrt(DL)) @ UL.T @ np.diag(sBA)
                  @ UR @ np.diag(np.sqrt(DR)))
    UBA, stemp, VhBA = LA.svd(weighted_mat, full_matrices=False)
    sBA = stemp / LA.norm(stemp)

    # build x,y gauge change matrices, implement gauge change on A and B
    x = np.conj(UL) @ np.diag(1 / np.sqrt(DL)) @ UBA
    y = np.conj(UR) @ np.diag(1 / np.sqrt(DR)) @ VhBA.T
    A = ncon([y, A], [[1, -1], [1, -2, -3]])
    B = ncon([B, x], [[-1, -2, 2], [2, -3]])

    return B, sBA, A

Define the Hamiltonian.

In [39]:
from scipy.linalg import expm

# define a Hamiltonian (here the quantum XX model)
sX = np.array([[0, 1], [1, 0]])
sY = np.array([[0, -1j], [1j, 0]])
hamAB = (np.real(np.kron(sX, sX) + np.kron(sY, sY))).reshape(2, 2, 2, 2)
hamBA = hamAB

# exponentiate Hamiltonian
tau = 0.1  # set time-step
evotype = 'imag'  # set imaginary time evolution
if evotype == "real":
    gateAB = expm(1j * tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
    gateBA = expm(1j * tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)
elif evotype == "imag":
    gateAB = expm(-tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
    gateBA = expm(-tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)

In [48]:
gateAB

array([[[[ 1.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  1.02006676],
         [-0.201336  ,  0.        ]]],


       [[[ 0.        , -0.201336  ],
         [ 1.02006676,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  1.        ]]]])

In [97]:
def apply_gate_MPS(gateAB, A, sAB, B, sBA, chi, stol=1e-7):
    """ apply a gate to an MPS across and a A-B link. Truncate the MPS back to
    some desired dimension chi"""

    # ensure singular values are above tolerance threshold
    sBA_trim = sBA * (sBA > stol) + stol * (sBA < stol)

    # contract gate into the MPS, then deompose composite tensor with SVD
    d = A.shape[1]
    chiBA = sBA_trim.shape[0]
    tensors = [np.diag(sBA_trim), A, np.diag(sAB), B, np.diag(sBA_trim), gateAB]
    connects = [[-1, 1], [1, 5, 2], [2, 4], [4, 6, 3], [3, -4], [-2, -3, 5, 6]]
    nshape = [d * chiBA, d * chiBA]
    utemp, stemp, vhtemp = LA.svd(ncon(tensors, connects).reshape(nshape),
                                full_matrices=False)

    # truncate to reduced dimension
    chitemp = min(chi, len(stemp))
    utemp = utemp[:, range(chitemp)].reshape(sBA_trim.shape[0], d * chitemp)
    vhtemp = vhtemp[range(chitemp), :].reshape(chitemp * d, chiBA)

    # remove environment weights to form new MPS tensors A and B
    A = (np.diag(1 / sBA_trim) @ utemp).reshape(sBA_trim.shape[0], d, chitemp)
    B = (vhtemp @ np.diag(1 / sBA_trim)).reshape(chitemp, d, chiBA)

    # new weights
    sAB = stemp[range(chitemp)] / LA.norm(stemp[range(chitemp)])

    return A, sAB, B

Putting it all together:

In [99]:
import numpy as np
from numpy import linalg as LA
from scipy.linalg import expm
from scipy.sparse.linalg import LinearOperator, eigs
from ncon import ncon
from typing import Optional


def doTEBD(hamAB: np.ndarray,
           hamBA: np.ndarray,
           A: np.ndarray,
           B: np.ndarray,
           sAB: np.ndarray,
           sBA: np.ndarray,
           chi: int,
           tau: float,
           evotype: Optional[str] = 'imag',
           numiter: Optional[int] = 1000,
           midsteps: Optional[int] = 10,
           E0: Optional[float] = 0.0):
    """
    Implementation of time evolution (real or imaginary) for MPS with 2-site unit
    cell (A-B), based on TEBD algorithm.
    Args:
    hamAB: nearest neighbor Hamiltonian coupling for A-B sites.
    hamBA: nearest neighbor Hamiltonian coupling for B-A sites.
    A: MPS tensor for A-sites of lattice.
    B: MPS tensor for B-sites of lattice.
    sAB: vector of weights for A-B links.
    sBA: vector of weights for B-A links.
    chi: maximum bond dimension of MPS.
    tau: time-step of evolution.
    evotype: set real (evotype='real') or imaginary (evotype='imag') evolution.
    numiter: number of time-step iterations to take.
    midsteps: number of time-steps between re-orthogonalization of the MPS.
    E0: specify the ground energy (if known).
    Returns:
    np.ndarray: MPS tensor for A-sites;
    np.ndarray: MPS tensor for B-sites;
    np.ndarray: vector sAB of weights for A-B links.
    np.ndarray: vector sBA of weights for B-A links.
    np.ndarray: two-site reduced density matrix rhoAB for A-B sites
    np.ndarray: two-site reduced density matrix rhoAB for B-A sites
    """
    # exponentiate Hamiltonian
    d = A.shape[1]
    if evotype == "real":
        gateAB = expm(1j * tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
        gateBA = expm(1j * tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)
    elif evotype == "imag":
        gateAB = expm(-tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
        gateBA = expm(-tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)

    # initialize environment matrices
    sigBA = np.eye(A.shape[0]) / A.shape[0]
    muAB = np.eye(A.shape[2]) / A.shape[2]

    for k in range(numiter + 1):
        if np.mod(k, midsteps) == 0 or (k == numiter):
            """ Bring MPS to normal form """

            # contract MPS from left and right
            sigBA, sigAB = left_contract_MPS(sigBA, sBA, A, sAB, B)
            muAB, muBA = right_contract_MPS(muAB, sBA, A, sAB, B)

            # orthogonalise A-B and B-A links
            B, sBA, A = orthog_MPS(sigBA, muBA, B, sBA, A)
            A, sAB, B = orthog_MPS(sigAB, muAB, A, sAB, B)

            # normalize the MPS tensors
            A_norm = np.sqrt(ncon([np.diag(sBA**2), A, np.conj(A), np.diag(sAB**2)],
                            [[1, 3], [1, 4, 2], [3, 4, 5], [2, 5]]))
            A = A / A_norm
            B_norm = np.sqrt(ncon([np.diag(sAB**2), B, np.conj(B), np.diag(sBA**2)],
                            [[1, 3], [1, 4, 2], [3, 4, 5], [2, 5]]))
            B = B / B_norm

            """ Compute energy and display """

            # compute 2-site local reduced density matrices
            rhoAB, rhoBA = loc_density_MPS(A, sAB, B, sBA)

            # evaluate the energy
            energyAB = ncon([hamAB, rhoAB], [[1, 2, 3, 4], [1, 2, 3, 4]])
            energyBA = ncon([hamBA, rhoBA], [[1, 2, 3, 4], [1, 2, 3, 4]])
            energy = 0.5 * (energyAB + energyBA)

            chitemp = min(A.shape[0], B.shape[0])
            enDiff = energy - E0
            print('iteration: %d of %d, chi: %d, t-step: %f, energy: %f, '
                'energy error: %e' % (k, numiter, chitemp, tau, energy, enDiff))

        """ Do evolution of MPS through one time-step """
        if k < numiter:
            # apply gate to A-B link
            A, sAB, B = apply_gate_MPS(gateAB, A, sAB, B, sBA, chi)

            # apply gate to B-A link
            B, sBA, A = apply_gate_MPS(gateBA, B, sBA, A, sAB, chi)

    rhoAB, rhoBA = loc_density_MPS(A, sAB, B, sBA)
    
    return A, B, sAB, sBA, rhoAB, rhoBA


def left_contract_MPS(sigBA, sBA, A, sAB, B):
    """ Contract an infinite 2-site unit cell from the left for the environment
    density matrices sigBA (B-A link) and sigAB (A-B link)"""

    # initialize the starting vector
    chiBA = A.shape[0]
    if sigBA.shape[0] == chiBA:
        v0 = sigBA.reshape(np.prod(sigBA.shape))
    else:
        v0 = (np.eye(chiBA) / chiBA).reshape(chiBA**2)

    # define network for transfer operator contract
    tensors = [np.diag(sBA), np.diag(sBA), A, A.conj(), np.diag(sAB),
             np.diag(sAB), B, B.conj()]
    labels = [[1, 2], [1, 3], [2, 4], [3, 5, 6], [4, 5, 7], [6, 8], [7, 9],
            [8, 10, -1], [9, 10, -2]]

    # define function for boundary contraction and pass to eigs
    def left_iter(sigBA):
        return ncon([sigBA.reshape([chiBA, chiBA]), *tensors],
                    labels).reshape([chiBA**2, 1])
    Dtemp, sigBA = eigs(LinearOperator((chiBA**2, chiBA**2), matvec=left_iter),
                      k=1, which='LM', v0=v0, tol=1e-10)

    # normalize the environment density matrix sigBA
    if np.isrealobj(A):
        sigBA = np.real(sigBA)
    sigBA = sigBA.reshape(chiBA, chiBA)
    sigBA = 0.5 * (sigBA + np.conj(sigBA.T))
    sigBA = sigBA / np.trace(sigBA)

    # compute density matric sigAB for A-B link
    sigAB = ncon([sigBA, np.diag(sBA), np.diag(sBA), A, np.conj(A)],
               [[1, 2], [1, 3], [2, 4], [3, 5, -1], [4, 5, -2]])
    sigAB = sigAB / np.trace(sigAB)

    return sigBA, sigAB


def right_contract_MPS(muAB, sBA, A, sAB, B):
    """ Contract an infinite 2-site unit cell from the right for the environment
    density matrices muAB (A-B link) and muBA (B-A link)"""

    # initialize the starting vector
    chiAB = A.shape[2]
    if muAB.shape[0] == chiAB:
        v0 = muAB.reshape(np.prod(muAB.shape))
    else:
        v0 = (np.eye(chiAB) / chiAB).reshape(chiAB**2)

    # define network for transfer operator contract
    tensors = [np.diag(sAB), np.diag(sAB), A, A.conj(), np.diag(sBA),
             np.diag(sBA), B, B.conj()]
    labels = [[1, 2], [3, 1], [5, 2], [6, 4, 3], [7, 4, 5], [8, 6], [10, 7],
            [-1, 9, 8], [-2, 9, 10]]

    # define function for boundary contraction and pass to eigs
    def right_iter(muAB):
        return ncon([muAB.reshape([chiAB, chiAB]), *tensors],
                    labels).reshape([chiAB**2, 1])
    Dtemp, muAB = eigs(LinearOperator((chiAB**2, chiAB**2), matvec=right_iter),
                     k=1, which='LM', v0=v0, tol=1e-10)

    # normalize the environment density matrix muAB
    if np.isrealobj(A):
        muAB = np.real(muAB)
    muAB = muAB.reshape(chiAB, chiAB)
    muAB = 0.5 * (muAB + np.conj(muAB.T))
    muAB = muAB / np.trace(muAB)

    # compute density matrix muBA for B-A link
    muBA = ncon([muAB, np.diag(sAB), np.diag(sAB), A, A.conj()],
              [[1, 2], [3, 1], [5, 2], [-1, 4, 3], [-2, 4, 5]])
    muBA = muBA / np.trace(muBA)

    return muAB, muBA


def orthog_MPS(sigBA, muBA, B, sBA, A, dtol=1e-12):
    """ set the MPS gauge across B-A link to the canonical form """
    # diagonalize left environment matrix
    dtemp, utemp = LA.eigh(sigBA)
    chitemp = sum(dtemp > dtol)
    DL = dtemp[range(-1, -chitemp - 1, -1)]
    UL = utemp[:, range(-1, -chitemp - 1, -1)]

    # diagonalize right environment matrix
    dtemp, utemp = LA.eigh(muBA)
    chitemp = sum(dtemp > dtol)
    DR = dtemp[range(-1, -chitemp - 1, -1)]
    UR = utemp[:, range(-1, -chitemp - 1, -1)]

    # compute new weights for B-A link
    weighted_mat = (np.diag(np.sqrt(DL)) @ UL.T @ np.diag(sBA)
                  @ UR @ np.diag(np.sqrt(DR)))
    UBA, stemp, VhBA = LA.svd(weighted_mat, full_matrices=False)
    sBA = stemp / LA.norm(stemp)

    # build x,y gauge change matrices, implement gauge change on A and B
    x = np.conj(UL) @ np.diag(1 / np.sqrt(DL)) @ UBA
    y = np.conj(UR) @ np.diag(1 / np.sqrt(DR)) @ VhBA.T
    A = ncon([y, A], [[1, -1], [1, -2, -3]])
    B = ncon([B, x], [[-1, -2, 2], [2, -3]])

    return B, sBA, A


def apply_gate_MPS(gateAB, A, sAB, B, sBA, chi, stol=1e-7):
    """ apply a gate to an MPS across and a A-B link. Truncate the MPS back to
    some desired dimension chi"""

    # ensure singular values are above tolerance threshold
    sBA_trim = sBA * (sBA > stol) + stol * (sBA < stol)

    # contract gate into the MPS, then deompose composite tensor with SVD
    d = A.shape[1]
    chiBA = sBA_trim.shape[0]
    tensors = [np.diag(sBA_trim), A, np.diag(sAB), B, np.diag(sBA_trim), gateAB]
    connects = [[-1, 1], [1, 5, 2], [2, 4], [4, 6, 3], [3, -4], [-2, -3, 5, 6]]
    nshape = [d * chiBA, d * chiBA]
    utemp, stemp, vhtemp = LA.svd(ncon(tensors, connects).reshape(nshape),
                                full_matrices=False)

    # truncate to reduced dimension
    chitemp = min(chi, len(stemp))
    utemp = utemp[:, range(chitemp)].reshape(sBA_trim.shape[0], d * chitemp)
    vhtemp = vhtemp[range(chitemp), :].reshape(chitemp * d, chiBA)

    # remove environment weights to form new MPS tensors A and B
    A = (np.diag(1 / sBA_trim) @ utemp).reshape(sBA_trim.shape[0], d, chitemp)
    B = (vhtemp @ np.diag(1 / sBA_trim)).reshape(chitemp, d, chiBA)

    # new weights
    sAB = stemp[range(chitemp)] / LA.norm(stemp[range(chitemp)])

    return A, sAB, B


def loc_density_MPS(A, sAB, B, sBA):
    """ Compute the local reduced density matrices from an MPS (assumend to be
    in canonical form)."""

    # recast singular weights into a matrix
    mAB = np.diag(sAB)
    mBA = np.diag(sBA)

    # contract MPS for local reduced density matrix (A-B)
    tensors = [np.diag(sBA**2), A, A.conj(), mAB, mAB, B, B.conj(),
             np.diag(sBA**2)]
    connects = [[3, 4], [3, -3, 1], [4, -1, 2], [1, 7], [2, 8], [7, -4, 5],
              [8, -2, 6], [5, 6]]
    rhoAB = ncon(tensors, connects)

    # contract MPS for local reduced density matrix (B-A)
    tensors = [np.diag(sAB**2), B, B.conj(), mBA, mBA, A, A.conj(),
             np.diag(sAB**2)]
    connects = [[3, 4], [3, -3, 1], [4, -1, 2], [1, 7], [2, 8], [7, -4, 5],
              [8, -2, 6], [5, 6]]
    rhoBA = ncon(tensors, connects)

    return rhoAB, rhoBA

In [100]:
import numpy as np
from ncon import ncon

""" Example 1: XX model """

# set bond dimensions and simulation options
chi = 16  # bond dimension
tau = 0.1  # timestep

numiter = 500  # number of timesteps
evotype = "imag"  # real or imaginary time evolution
E0 = -4 / np.pi  # specify exact ground energy (if known)
midsteps = int(1 / tau)  # timesteps between MPS re-orthogonalization

# define Hamiltonian (quantum XX model)
sX = np.array([[0, 1], [1, 0]])
sY = np.array([[0, -1j], [1j, 0]])
sZ = np.array([[1, 0], [0, -1]])
hamAB = (np.real(np.kron(sX, sX) + np.kron(sY, sY))).reshape(2, 2, 2, 2)
hamBA = (np.real(np.kron(sX, sX) + np.kron(sY, sY))).reshape(2, 2, 2, 2)

# initialize tensors
d = hamAB.shape[0]
sAB = np.ones(chi) / np.sqrt(chi)
sBA = np.ones(chi) / np.sqrt(chi)
A = np.random.rand(chi, d, chi)
B = np.random.rand(chi, d, chi)

""" Imaginary time evolution with TEBD """
# run TEBD routine
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# continute running TEBD routine with reduced timestep
tau = 0.01
numiter = 2000
midsteps = 100
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# continute running TEBD routine with reduced timestep and increased bond dim
chi = 32
tau = 0.001
numiter = 20000
midsteps = 1000
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# compare with exact results
energyMPS = np.real(0.5 * ncon([hamAB, rhoAB], [[1, 2, 3, 4], [1, 2, 3, 4]]) +
                    0.5 * ncon([hamBA, rhoBA], [[1, 2, 3, 4], [1, 2, 3, 4]]))
enErr = abs(energyMPS - E0)
print('Final results => Bond dim: %d, Energy: %f, Energy Error: %e' %
      (chi, energyMPS, enErr))

iteration: 0 of 500, chi: 16, t-step: 0.100000, energy: 1.000638, energy error: 2.273877e+00
iteration: 10 of 500, chi: 16, t-step: 0.100000, energy: -1.251253, energy error: 2.198659e-02
iteration: 20 of 500, chi: 16, t-step: 0.100000, energy: -1.262697, energy error: 1.054270e-02
iteration: 30 of 500, chi: 16, t-step: 0.100000, energy: -1.263952, energy error: 9.287530e-03
iteration: 40 of 500, chi: 16, t-step: 0.100000, energy: -1.264334, energy error: 8.905149e-03
iteration: 50 of 500, chi: 16, t-step: 0.100000, energy: -1.264486, energy error: 8.753174e-03
iteration: 60 of 500, chi: 16, t-step: 0.100000, energy: -1.264556, energy error: 8.683885e-03
iteration: 70 of 500, chi: 16, t-step: 0.100000, energy: -1.264590, energy error: 8.649572e-03
iteration: 80 of 500, chi: 16, t-step: 0.100000, energy: -1.264608, energy error: 8.631646e-03
iteration: 90 of 500, chi: 16, t-step: 0.100000, energy: -1.264618, energy error: 8.621948e-03
iteration: 100 of 500, chi: 16, t-step: 0.100000, en

iteration: 13000 of 20000, chi: 32, t-step: 0.001000, energy: -1.273171, energy error: 6.889929e-05
iteration: 14000 of 20000, chi: 32, t-step: 0.001000, energy: -1.273171, energy error: 6.817975e-05
iteration: 15000 of 20000, chi: 32, t-step: 0.001000, energy: -1.273172, energy error: 6.763113e-05
iteration: 16000 of 20000, chi: 32, t-step: 0.001000, energy: -1.273172, energy error: 6.721153e-05
iteration: 17000 of 20000, chi: 32, t-step: 0.001000, energy: -1.273173, energy error: 6.688975e-05
iteration: 18000 of 20000, chi: 32, t-step: 0.001000, energy: -1.273173, energy error: 6.664240e-05
iteration: 19000 of 20000, chi: 32, t-step: 0.001000, energy: -1.273173, energy error: 6.645188e-05
iteration: 20000 of 20000, chi: 32, t-step: 0.001000, energy: -1.273173, energy error: 6.630489e-05
Final results => Bond dim: 32, Energy: -1.273173, Energy Error: 6.630489e-05
